In [1]:
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from src.model import Model
from src import utils
from src.pytorch_utils import utils as ptu
from config import cfg
import src.data_loader as dl

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
model_epoch = 'best'
if cfg.load is not None and os.path.exists(os.path.join(cfg.models_dir, cfg.version, ptu.naming_scheme(cfg.version, epoch=model_epoch)) + '.pth'):
    print(f'Loads {cfg.version}')
    checkpoint = ptu.load_model(device, version=cfg.version, models_dir=cfg.models_dir, epoch=model_epoch)
    if cfg.prints == 'display':
        display(checkpoint.log.sort_index(ascending=False).head(20))
    elif cfg.prints == 'print':
        print(checkpoint.log.sort_index(ascending=False).head(20))
else:
    model = Model(backbone=cfg.backbone, num_classes=cfg.num_classes)
    model.to(device)
    
    if cfg.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.fc.parameters(),
                                    lr=cfg.lr,
                                    momentum=cfg.optimizer_momentum,
                                    weight_decay=cfg.wd)
    else:
        optimizer = torch.optim.Adam(model.fc.parameters(),
                                    lr=cfg.lr,
                                    weight_decay=cfg.wd)
    
    criterion = nn.CrossEntropyLoss().to(device)
    
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                              T_max=cfg.epochs,
                                                              eta_min=cfg.min_lr)
    
    checkpoint = utils.Checkpoint(version=cfg.version,
                                  model=model,
                                  optimizer=optimizer,
                                  lr_scheduler=lr_scheduler,
                                  criterion=criterion,
                                  score=utils.accuracy_score,
                                  models_dir=cfg.models_dir,
                                  best_policy=cfg.best_policy,
                                  save=cfg.save,
                                 )

Loads resnet50_sgd_bs64


,train_time,timestamp,train_loss,val_loss,train_score,val_score,batch_size,best,dampening,initial_lr,lr,momentum,nesterov,weight_decay
299,0.787718,20:22:10 22-08-2021,1.420964,1.426026,0.525541,0.528245,64,True,0.0,0.0001,0.000051,0.9,0.0,0.0005
298,0.782635,20:21:21 22-08-2021,1.443495,1.452435,0.526342,0.523438,64,False,0.0,0.0001,0.000051,0.9,0.0,0.0005
297,0.789208,20:20:31 22-08-2021,1.434631,1.453963,0.522035,0.519030,64,False,0.0,0.0001,0.000051,0.9,0.0,0.0005
296,0.776283,20:19:42 22-08-2021,1.424582,1.439210,0.525441,0.524239,64,False,0.0,0.0001,0.000051,0.9,0.0,0.0005
295,0.777388,20:18:53 22-08-2021,1.451171,1.463350,0.524339,0.517829,64,False,0.0,0.0001,0.000052,0.9,0.0,0.0005
294,0.772217,20:18:04 22-08-2021,1.446204,1.462432,0.524339,0.519832,64,False,0.0,0.0001,0.000052,0.9,0.0,0.0005
293,0.756187,20:17:15 22-08-2021,1.420360,1.434866,0.531951,0.525441,64,False,0.0,0.0001,0.000052,0.9,0.0,0.0005
292,0.782749,20:16:27 22-08-2021,1.444274,1.443484,0.521935,0.521334,64,False,0.0,0.0001,0.000052,0.9,0.0,0.0005
291,0.774950,20:15:38 22-08-2021,1.436122,1.442689,0.520232,0.518830,64,False,0.0,0.0001,0.000053,0.9,0.0,0.0005
290,0.776757,20:14:49 22-08-2021,1.435289,1.449120,0.522536,0.517929,64,False,0.0,0.0001,0.000053,0.9,0.0,0.0005


In [4]:
train_loader, train_eval_loader = dl.train_eval_loaders_cifar10(data_dir=cfg.data_dir, batch_size=cfg.bs)
test_loader = dl.test_loader(data_dir=cfg.data_dir, batch_size=cfg.bs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# checkpoint.train(train_loader=train_loader,
#                  train_eval_loader=train_eval_loader,
#                  val_loader=test_loader,
#                  train_epochs=int(max(0, cfg.epochs - checkpoint.get_log())),
#                  optimizer_params=cfg.optimizer_params,
#                  prints=cfg.prints,
#                  epochs_save=cfg.epochs_save,
#                  epochs_evaluate_train=cfg.epochs_evaluate_train,
#                  epochs_evaluate_validation=cfg.epochs_evaluate_validation,
#                  max_iterations_train=cfg.max_iterations,
#                  max_iterations_val=cfg.max_iterations,
#                  device=device,
#                  tqdm_bar=cfg.tqdm_bar,
#                  save=cfg.save,
#                  save_log=cfg.save_log,
#                 )

In [17]:
df = pd.DataFrame(columns=['augment', 'angle', 'loss', 'score'])

In [18]:
for angle in range(0, 91, 10):
    print(f'Angle {angle}')
    test_loader = dl.test_loader(data_dir=cfg.data_dir,
                                 batch_size=cfg.bs,
                                 augment=True,
                                 angle=angle)
    loss, score, results = checkpoint.evaluate(loader=test_loader,
                                               device=device,
                                               tqdm_bar=True)
    df = df.append({'augment': 'rotation', 'angle': angle, 'loss': loss, 'score': score},
                   ignore_index=True)

Angle 0
Files already downloaded and verified


epoch=300: 100%|██████████| 156/156 [00:04<00:00, 33.13it/s, loss=1.58, avg_loss=1.43, score=0.5]  


Angle 10
Files already downloaded and verified


epoch=300: 100%|██████████| 156/156 [00:04<00:00, 32.99it/s, loss=1.88, avg_loss=1.75, score=0.391]


Angle 20
Files already downloaded and verified


...

epoch=300: 100%|██████████| 156/156 [00:04<00:00, 33.01it/s, loss=2.86, avg_loss=2.62, score=0.25] 


Angle 80
Files already downloaded and verified


epoch=300: 100%|██████████| 156/156 [00:04<00:00, 32.64it/s, loss=2.51, avg_loss=2.63, score=0.328]


Angle 90
Files already downloaded and verified


epoch=300: 100%|██████████| 156/156 [00:04<00:00, 32.86it/s, loss=2.82, avg_loss=2.61, score=0.203]


In [19]:
df

,augment,angle,loss,score
0,rotation,0,1.426026,0.528245
1,rotation,10,1.750166,0.431991
2,rotation,20,2.063187,0.357372
3,rotation,30,2.268319,0.314904
4,rotation,40,2.419700,0.294471
5,rotation,50,2.546118,0.270132
6,rotation,60,2.603346,0.265325
7,rotation,70,2.622925,0.261919
8,rotation,80,2.626683,0.251703
9,rotation,90,2.606750,0.260016
